In [1]:
from qiskit import *
import math
import numpy as np
import hashlib
import hmac
import time
import json
from Crypto.Cipher import AES
from Crypto.Random import get_random_bytes
import Padding

In [2]:
import qiskit
qiskit.__qiskit_version__

{'qiskit': '0.45.0', 'qiskit-aer': '0.13.0', 'qiskit-ignis': None, 'qiskit-ibmq-provider': None, 'qiskit-nature': None, 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [3]:
simulator = Aer.get_backend('qasm_simulator')

In [9]:
class QChannel():
    def __init__(self, n, delta, c ,numberOfCommunicators):
        # size of INFO BITS
        self.n = n
        
        # delta parameter
        self.delta = delta

        # N
        self.N = math.ceil(8*self.n*(1+self.delta))

        # probability of not measuring by communicators
        self.c = c

        self.numberOfCommunicators = numberOfCommunicators

    def perfromAlgorithm(self):
        info = dict()

        for i in range(self.numberOfCommunicators):
            for j in range(i+1, self.numberOfCommunicators):
                info[(i, j)] = ""


    def performTrip(self):

        # quantum channel
        circuit = QuantumCircuit(self.N, self.N)

        # encoding of initial state
        # 0 = |+> and 1 = |->
        initialStateDecision = np.random.randint(2, size=self.N)

        print("initialStateDecision")
        print(initialStateDecision)

        for i in range(len(initialStateDecision)):
            if initialStateDecision[i]:
                circuit.x(i)
            circuit.h(i)

        circuit.barrier()

        # decisions by each communicator

        allMeasureDecisions = []

        for i in range(self.numberOfCommunicators):
            allMeasureDecisions.append(list(np.random.choice([0, 1], self.N, p=[self.c, 1-self.c])))
        
        print("allMeasureDecisions")
        for amd in allMeasureDecisions:
            print(amd)

        # communicator result

        values = []

        for i in range(self.numberOfCommunicators):
            for j in range(self.N):
                if allMeasureDecisions[i][j]:
                    circuit.measure(j, j)
            result = execute(circuit, backend=simulator, shots=1).result()
            count = result.get_counts(circuit)
            values.append([int(x) for x in list(list(count.keys())[0][::-1])])
            for j in range(len(values[-1])):
                if allMeasureDecisions[i][j]:
                    if  values[-1][j] == 0:
                        circuit.initialize([1, 0], j)
                    else:
                        circuit.initialize([0, 1], j)
            circuit.barrier()

        print("values")
        for v in values:
            print(v)

        # (p0) None of Bi measured the travel qubit
        # (p1) One of Bi measured the travel qubit but the other did not
        # (p2) Two of Bi measured the travel qubit.
        # (p3) Three or more of Bi measured the travel qubit.

        partiesInvolved = [[] for i in range(self.N)]

        for j in range(self.N):
            for i in range(self.numberOfCommunicators):
                if allMeasureDecisions[i][j]:
                    partiesInvolved[j].append(i)

        print("partiesInvolved")
        print(partiesInvolved)

        cases = []

        for i in range(self.N):
            cases.append(len(partiesInvolved[i]))

        print("cases")
        print(cases)
        
        ctrlA = set()
        ctrlB = set()
        sift = []

        for i in range(len(cases)):
            if cases[i] == 0:
                ctrlA.add(i)
            elif cases[i] == 1:
                ctrlB.add(i)
            elif cases[i] == 2:
                sift.append(i)

        print(f'{ctrlA=}')
        print(f'{ctrlB=}')
        print(f'{sift=}')

        if len(sift) < 3:
            print("Abroted: len(siftAble) < "+str(self.n + 1))
            return None

        # decisions of measure basis by channel
        # 0 = Z and 1 = H

        finalMeasurementBasis = np.random.randint(2, size=self.N)

        for i in range(self.N):
            if finalMeasurementBasis[i]:
                circuit.h(i)
            circuit.measure(i, i)

        print("finalMeasurementBasis")
        print(finalMeasurementBasis)

        result2 = execute(circuit, backend=simulator, shots=1).result()
        count2 = result2.get_counts(circuit)
        value2 = [int(x) for x in list(list(count2.keys())[0][::-1])]

        print("value2")
        print(value2)
        
        # get info bits

        info = sift[2:]

        siftCheck = set(sift[:2])

        print(f'{info=}')
        print(f'{siftCheck=}')

        # checkiing error

        qberCtrlA = 0
        qberCtrlB = 0
        qberSift = 0

        for q in ctrlA:
            if finalMeasurementBasis[q] == 1:
                qberCtrlA += int(value2[q] != initialStateDecision[q])
        
        for q in ctrlB:
            if finalMeasurementBasis[q] == 0:
                qberCtrlB += int(value2[q] != values[partiesInvolved[q][0]][q])


        for q in sift:
            qberSift += int(values[partiesInvolved[q][0]][q] != values[partiesInvolved[q][1]][q])

        # qberCtrl = (qberCtrlA+qberCtrlB)/(len(ctrlA)+len(ctrlB))
        # qberSift = qberSift/len(sift)

        print(f'{qberCtrlA=}')
        print(f'{qberCtrlB=}')

        # print(f'{qberCtrl=}')
        print(f'{qberSift=}')

        print(circuit.draw())

In [10]:
q = QChannel(n=2, delta=1/8, c=0.7, numberOfCommunicators=3)

In [12]:
q.performTrip()

initialStateDecision
[1 0 0 0 0 1 1 1 0 0 1 0 1 1 1 0 1 0]
allMeasureDecisions
[1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0]
[1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0]
[0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0]
values
[0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0]
[0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0]
partiesInvolved
[[0, 1], [1, 2], [], [1], [], [0], [0], [], [0], [0, 1], [], [0, 1], [1], [0, 2], [], [], [0, 2], []]
cases
[2, 2, 0, 1, 0, 1, 1, 0, 1, 2, 0, 2, 1, 2, 0, 0, 2, 0]
ctrlA={2, 4, 7, 10, 14, 15, 17}
ctrlB={3, 5, 6, 8, 12}
sift=[0, 1, 9, 11, 13, 16]
finalMeasurementBasis
[1 0 0 0 1 0 0 1 1 0 1 0 1 1 0 1 0 1]
value2
[0, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0]
info=[9, 11, 13, 16]
siftCheck={0, 1}
qberCtrlA=0
qberCtrlB=0
qberSift=0
      ┌───┐┌───┐ ░ ┌─┐┌─────────────────┐                                      »
 q_0: ┤ X ├┤ H ├─░─┤M├┤ Initialize(1,0) ├──────